In [1]:
from sklearnex import patch_sklearn
patch_sklearn()

Intel(R) Extension for Scikit-learn* enabled (https://github.com/intel/scikit-learn-intelex)


In [10]:
import os
import sys; sys.path.insert(0, os.path.abspath("../"))
from sklearn.ensemble import GradientBoostingClassifier
import cv2
import xgboost as xgb
import numpy as np
from sklearn import svm
from sklearn.ensemble import AdaBoostClassifier
import pandas as pd
from tqdm import tqdm
from dataset.dataset import SkinLesion_Dataset, SegExamples
from pipeline.preprocessing import SkinLesionPreprocessing
from pipeline.feature_extraction import FeaturesExtraction

import multiprocessing as mp
from tqdm import tqdm

from sklearn.model_selection import GridSearchCV, StratifiedKFold
from sklearn.ensemble import RandomForestClassifier
import pickle
from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.metrics import f1_score, accuracy_score
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import GridSearchCV, StratifiedKFold
from sklearn.ensemble import RandomForestClassifier
import pickle
from sklearn.svm import LinearSVC
from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.metrics import f1_score, accuracy_score, make_scorer, cohen_kappa_score, roc_auc_score, balanced_accuracy_score
from sklearn.feature_selection import VarianceThreshold
from sklearn.feature_selection import SelectFromModel
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from boruta import BorutaPy
from sklearn.neighbors import KNeighborsClassifier

In [3]:
train_df_1 = pd.read_feather("../data/binary/train_all_feat.f")
train_df_2 = pd.read_feather("../data/binary/train_color_local_feat.f")
val_df_1 = pd.read_feather("../data/binary/val_all_features.f")
val_df_2 = pd.read_feather("../data/binary/val_color_local_feat.f")

train_df = pd.concat([train_df_2.iloc[:,:60], train_df_1], axis=1)
val_df = pd.concat([val_df_2.iloc[:, :60], val_df_1], axis=1)

train_shape = train_df.shape
val_shape = val_df.shape

train_df.replace({'nevus': 1, 'others': 0}, inplace=True)
val_df.replace({'nevus': 1, 'others': 0}, inplace=True)

df_train = train_df.iloc[:train_shape[0], :].sample(frac=1, random_state=42)
df_val = val_df.iloc[:val_shape[0], :].sample(frac=1, random_state=42)
# df_val

X_train, y_train = df_train.iloc[:, :(train_shape[1]-1)].to_numpy(dtype=np.float32), df_train.iloc[:, (train_shape[1]-1)].to_numpy()
X_test, y_test = df_val.iloc[:, :(val_shape[1]-1)].to_numpy(dtype= np.float32), df_val.iloc[:, (val_shape[1]-1)].to_numpy()

print(f'X_train: {X_train.shape}, y_train: {y_train.shape}, X_test: {X_test.shape}, y_test: {y_test.shape}')
print(f'final class balance [TRAIN]: {np.unique(y_train, return_counts=True)}', f'[VAL]: {np.unique(y_test, return_counts=True)}')

X_train_f = pd.DataFrame(X_train)
X_test_f = pd.DataFrame(X_test)
y_train_f = pd.DataFrame(y_train)
y_test_f = pd.DataFrame(y_test)
frames = [X_train_f, X_test_f]
f = [y_train_f, y_test_f]
X = pd.concat(frames)
y = pd.concat(f)

# Feature Selection -- VARIANCE THRESHOLD
sel_var = VarianceThreshold(threshold=(.8 * (1 - .8)))
X_var= sel_var.fit_transform(X)
print(f'X_var: {X_var.shape}')

## Feature Selection -- SelectFromModel
sel_sfm = SelectFromModel(LinearSVC(C = 0.09, penalty="l1",  dual=False))
print("Performing Feature Selection ... ")
X_sfm = sel_sfm.fit_transform(X, y)
print(f'X_sfm: {X_sfm.shape}')

# split dataframe X again into train and test -- VARIANCE THRESHOLD
X_train_var = pd.DataFrame(X_var).iloc[:len(X_train_f[0])].to_numpy(dtype=np.float32)
X_test_var = pd.DataFrame(X_var).iloc[len(X_train_f[0]):].to_numpy(dtype=np.float32)
y_train_new = pd.DataFrame(y).iloc[:len(y_train_f[0])].to_numpy(dtype=np.float32)
y_test_new = pd.DataFrame(y).iloc[len(y_train_f[0]):].to_numpy(dtype=np.float32)
print(f'X_train_var: {X_train_var.shape}, y_train: {y_train.shape}, X_test_var: {X_test_var.shape}, y_test: {y_test.shape}')

# split dataframe X again into train and test SFM
X_train_new = pd.DataFrame(X_sfm).iloc[:len(X_train_f[0])].to_numpy(dtype=np.float32)
X_test_new = pd.DataFrame(X_sfm).iloc[len(X_train_f[0]):].to_numpy(dtype=np.float32)
y_train_new = pd.DataFrame(y).iloc[:len(y_train_f[0])].to_numpy(dtype=np.float32)
y_test_new = pd.DataFrame(y).iloc[len(y_train_f[0]):].to_numpy(dtype=np.float32)
print(f'X_train_new: {X_train_new.shape}, y_train: {y_train.shape}, X_test_new: {X_test_new.shape}, y_test: {y_test.shape}')

X_train: (15195, 203), y_train: (15195,), X_test: (3796, 203), y_test: (3796,)
final class balance [TRAIN]: (array([0, 1], dtype=int64), array([7470, 7725], dtype=int64)) [VAL]: (array([0, 1], dtype=int64), array([1865, 1931], dtype=int64))
X_var: (18991, 151)
Performing Feature Selection ... 


C:\Users\Lenovo\.virtualenvs\Skin_lesion_segmentation\lib\site-packages\sklearn\utils\validation.py:1111: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


X_sfm: (18991, 132)
X_train_var: (15195, 151), y_train: (15195,), X_test_var: (3796, 151), y_test: (3796,)
X_train_new: (15195, 132), y_train: (15195,), X_test_new: (3796, 132), y_test: (3796,)


C:\Users\Lenovo\.virtualenvs\Skin_lesion_segmentation\lib\site-packages\sklearn\svm\_base.py:1225: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


## Support Vector Classifier (SVC)

In [4]:
# Without Feature Selection
classifier = svm.SVC(C= 10, kernel= 'rbf')
pipe = Pipeline([('scaler', StandardScaler()),('classifier', classifier)])
pipe.fit(X_train, y_train)
y_train_predicted = pipe.predict(X_train)
y_test_predicted =  pipe.predict(X_test)
print('F1 Score of train data: %0.4f' %f1_score(y_train,y_train_predicted,average='macro'), f'Acc: {accuracy_score(y_train, y_train_predicted)}')
print('F1 Score of test data: %0.4f' %f1_score(y_test,y_test_predicted,average='macro'), f'Acc: {accuracy_score(y_test, y_test_predicted)}')

INFO:root:sklearn.svm.SVC.fit: running accelerated version on CPU
INFO:root:sklearn.svm.SVC.predict: running accelerated version on CPU
INFO:root:sklearn.svm.SVC.predict: running accelerated version on CPU


F1 Score of train data: 0.9359 Acc: 0.9359657782165186
F1 Score of test data: 0.8508 Acc: 0.850895679662803


In [5]:
# With Variance Thresholding Feature Selection
classifier = svm.SVC(C= 10, kernel= 'rbf')
pipe = Pipeline([('scaler', StandardScaler()),('classifier', classifier)])
pipe.fit(X_train_var, y_train_new)
y_train_predicted = pipe.predict(X_train_var)
y_test_predicted =  pipe.predict(X_test_var)
print('F1 Score of train data: %0.4f' %f1_score(y_train_new,y_train_predicted,average='macro'), f'Acc: {accuracy_score(y_train_new, y_train_predicted)}')
print('F1 Score of test data: %0.4f' %f1_score(y_test_new,y_test_predicted,average='macro'), f'Acc: {accuracy_score(y_test_new, y_test_predicted)}')

INFO:root:sklearn.svm.SVC.fit: running accelerated version on CPU
C:\Users\Lenovo\.virtualenvs\Skin_lesion_segmentation\lib\site-packages\onedal\datatypes\validation.py:127: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = _column_or_1d(y, warn=True)
INFO:root:sklearn.svm.SVC.predict: running accelerated version on CPU
INFO:root:sklearn.svm.SVC.predict: running accelerated version on CPU


F1 Score of train data: 0.9245 Acc: 0.9245804540967424
F1 Score of test data: 0.8416 Acc: 0.8416754478398314


In [6]:
# With Select from model method
classifier = svm.SVC(C= 10, kernel= 'rbf')
pipe = Pipeline([('scaler', StandardScaler()),('classifier', classifier)])
pipe.fit(X_train_new, y_train_new)
y_train_predicted = pipe.predict(X_train_new)
y_test_predicted =  pipe.predict(X_test_new)
print('F1 Score of train data: %0.4f' %f1_score(y_train_new,y_train_predicted,average='macro'), f'Acc: {accuracy_score(y_train_new, y_train_predicted)}')
print('F1 Score of test data: %0.4f' %f1_score(y_test_new,y_test_predicted,average='macro'), f'Acc: {accuracy_score(y_test_new, y_test_predicted)}')

INFO:root:sklearn.svm.SVC.fit: running accelerated version on CPU
C:\Users\Lenovo\.virtualenvs\Skin_lesion_segmentation\lib\site-packages\onedal\datatypes\validation.py:127: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = _column_or_1d(y, warn=True)
INFO:root:sklearn.svm.SVC.predict: running accelerated version on CPU
INFO:root:sklearn.svm.SVC.predict: running accelerated version on CPU


F1 Score of train data: 0.9257 Acc: 0.9256992431720961
F1 Score of test data: 0.8457 Acc: 0.8458904109589042


## XGBoost

In [7]:
# Default xgboost without gridsearch
classifier = xgb.XGBClassifier()
pipe = Pipeline([('scaler', StandardScaler()),('classifier', classifier)])
pipe.fit(X_train, y_train)
y_train_predicted = pipe.predict(X_train)
y_test_predicted =  pipe.predict(X_test)
print('F1 Score of train data: %0.4f' %f1_score(y_train,y_train_predicted,average='macro'), f'Acc: {accuracy_score(y_train, y_train_predicted)}')
print('F1 Score of test data: %0.4f' %f1_score(y_test,y_test_predicted,average='macro'), f'Acc: {accuracy_score(y_test, y_test_predicted)}')

F1 Score of train data: 0.9923 Acc: 0.9923000987166831
F1 Score of test data: 0.8373 Acc: 0.8374604847207587


In [8]:
# With Variance Thresholding Feature Selection
classifier = xgb.XGBClassifier()
pipe = Pipeline([('scaler', StandardScaler()),('classifier', classifier)])
pipe.fit(X_train_var, y_train_new)
y_train_predicted = pipe.predict(X_train_var)
y_test_predicted =  pipe.predict(X_test_var)
print('F1 Score of train data: %0.4f' %f1_score(y_train_new,y_train_predicted,average='macro'), f'Acc: {accuracy_score(y_train_new, y_train_predicted)}')
print('F1 Score of test data: %0.4f' %f1_score(y_test_new,y_test_predicted,average='macro'), f'Acc: {accuracy_score(y_test_new, y_test_predicted)}')

F1 Score of train data: 0.9895 Acc: 0.989470220467259
F1 Score of test data: 0.8239 Acc: 0.8240252897787145


In [9]:
# With Select from model method
classifier = xgb.XGBClassifier()
pipe = Pipeline([('scaler', StandardScaler()),('classifier', classifier)])
pipe.fit(X_train_new, y_train_new)
y_train_predicted = pipe.predict(X_train_new)
y_test_predicted =  pipe.predict(X_test_new)
print('F1 Score of train data: %0.4f' %f1_score(y_train_new,y_train_predicted,average='macro'), f'Acc: {accuracy_score(y_train_new, y_train_predicted)}')
print('F1 Score of test data: %0.4f' %f1_score(y_test_new,y_test_predicted,average='macro'), f'Acc: {accuracy_score(y_test_new, y_test_predicted)}')

F1 Score of train data: 0.9850 Acc: 0.984995064165844
F1 Score of test data: 0.8249 Acc: 0.8250790305584826


## KNN

In [17]:
# KNN deafult
classifier = KNeighborsClassifier()
pipe = Pipeline([('scaler', StandardScaler()),('classifier', classifier)])
pipe.fit(X_train, y_train)
y_train_predicted = pipe.predict(X_train)
y_test_predicted =  pipe.predict(X_test)
print('F1 Score of train data: %0.4f' %f1_score(y_train,y_train_predicted,average='macro'), f'Acc: {accuracy_score(y_train, y_train_predicted)}')
print('F1 Score of test data: %0.4f' %f1_score(y_test,y_test_predicted,average='macro'), f'Acc: {accuracy_score(y_test, y_test_predicted)}')

INFO:root:sklearn.neighbors.KNeighborsClassifier.fit: running accelerated version on CPU
INFO:root:sklearn.neighbors.KNeighborsClassifier.predict: running accelerated version on CPU
INFO:root:sklearn.neighbors.KNeighborsClassifier.predict: running accelerated version on CPU


F1 Score of train data: 0.8807 Acc: 0.8806844356696282
F1 Score of test data: 0.8132 Acc: 0.8132244467860906


## Adaboost

In [19]:
# Adaboost with the best grid search
classifier = AdaBoostClassifier(learning_rate= 0.1, n_estimators=500)
pipe = Pipeline([('scaler', StandardScaler()),('classifier', classifier)])
pipe.fit(X_train, y_train)
y_train_predicted = pipe.predict(X_train)
y_test_predicted =  pipe.predict(X_test)
print('F1 Score of train data: %0.4f' %f1_score(y_train,y_train_predicted,average='macro'), f'Acc: {accuracy_score(y_train, y_train_predicted)}')
print('F1 Score of test data: %0.4f' %f1_score(y_test,y_test_predicted,average='macro'), f'Acc: {accuracy_score(y_test, y_test_predicted)}')

F1 Score of train data: 0.8171 Acc: 0.8171108917407042
F1 Score of test data: 0.8084 Acc: 0.8084826132771338


## Logistic Regression

In [22]:
# LR Default
classifier = LogisticRegression(C= 3, penalty= 'l2')
pipe = Pipeline([('scaler', StandardScaler()),('classifier', classifier)])
pipe.fit(X_train, y_train)
y_train_predicted = pipe.predict(X_train)
y_test_predicted =  pipe.predict(X_test)
print('F1 Score of train data: %0.4f' %f1_score(y_train,y_train_predicted,average='macro'), f'Acc: {accuracy_score(y_train, y_train_predicted)}')
print('F1 Score of test data: %0.4f' %f1_score(y_test,y_test_predicted,average='macro'), f'Acc: {accuracy_score(y_test, y_test_predicted)}')

C:\Users\Lenovo\.virtualenvs\Skin_lesion_segmentation\lib\site-packages\daal4py\sklearn\linear_model\logistic_path.py:548: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
INFO:root:sklearn.linear_model.LogisticRegression.fit: running accelerated version on CPU
INFO:root:sklearn.linear_model.LogisticRegression.predict: running accelerated version on CPU
INFO:root:sklearn.linear_model.LogisticRegression.predict: running accelerated version on CPU


F1 Score of train data: 0.8097 Acc: 0.8097400460677855
F1 Score of test data: 0.8094 Acc: 0.809536354056902


## Random Forest

In [23]:
# RF default
classifier = RandomForestClassifier()
pipe = Pipeline([('scaler', StandardScaler()),('classifier', classifier)])
pipe.fit(X_train, y_train)
y_train_predicted = pipe.predict(X_train)
y_test_predicted =  pipe.predict(X_test)
print('F1 Score of train data: %0.4f' %f1_score(y_train,y_train_predicted,average='macro'), f'Acc: {accuracy_score(y_train, y_train_predicted)}')
print('F1 Score of test data: %0.4f' %f1_score(y_test,y_test_predicted,average='macro'), f'Acc: {accuracy_score(y_test, y_test_predicted)}')

INFO:root:sklearn.ensemble.RandomForestClassifier.fit: running accelerated version on CPU
INFO:root:sklearn.ensemble.RandomForestClassifier.predict: running accelerated version on CPU
INFO:root:sklearn.ensemble.RandomForestClassifier.predict: running accelerated version on CPU


F1 Score of train data: 1.0000 Acc: 1.0
F1 Score of test data: 0.8303 Acc: 0.8303477344573235


In [24]:
# RF with grid search result
classifier = RandomForestClassifier(max_depth= 50, n_estimators=1000)
pipe = Pipeline([('scaler', StandardScaler()),('classifier', classifier)])
pipe.fit(X_train, y_train)
y_train_predicted = pipe.predict(X_train)
y_test_predicted =  pipe.predict(X_test)
print('F1 Score of train data: %0.4f' %f1_score(y_train,y_train_predicted,average='macro'), f'Acc: {accuracy_score(y_train, y_train_predicted)}')
print('F1 Score of test data: %0.4f' %f1_score(y_test,y_test_predicted,average='macro'), f'Acc: {accuracy_score(y_test, y_test_predicted)}')

INFO:root:sklearn.ensemble.RandomForestClassifier.fit: running accelerated version on CPU
INFO:root:sklearn.ensemble.RandomForestClassifier.predict: running accelerated version on CPU
INFO:root:sklearn.ensemble.RandomForestClassifier.predict: running accelerated version on CPU


F1 Score of train data: 1.0000 Acc: 1.0
F1 Score of test data: 0.8331 Acc: 0.833245521601686


## Gradient Boosting

In [25]:
# Gradient boosting default values
classifier =GradientBoostingClassifier()
pipe = Pipeline([('scaler', StandardScaler()),('classifier', classifier)])
pipe.fit(X_train, y_train)
y_train_predicted = pipe.predict(X_train)
y_test_predicted =  pipe.predict(X_test)
print('F1 Score of train data: %0.4f' %f1_score(y_train,y_train_predicted,average='macro'), f'Acc: {accuracy_score(y_train, y_train_predicted)}')
print('F1 Score of test data: %0.4f' %f1_score(y_test,y_test_predicted,average='macro'), f'Acc: {accuracy_score(y_test, y_test_predicted)}')

F1 Score of train data: 0.8398 Acc: 0.8398815399802566
F1 Score of test data: 0.8102 Acc: 0.8103266596417281


In [27]:
# Gradient boosting grid search values

classifier =GradientBoostingClassifier(learning_rate =0.1, loss = 'log_loss', n_estimators = 200)
pipe = Pipeline([('scaler', StandardScaler()),('classifier', classifier)])
pipe.fit(X_train, y_train)
y_train_predicted = pipe.predict(X_train)
y_test_predicted =  pipe.predict(X_test)
print('F1 Score of train data: %0.4f' %f1_score(y_train,y_train_predicted,average='macro'), f'Acc: {accuracy_score(y_train, y_train_predicted)}')
print('F1 Score of test data: %0.4f' %f1_score(y_test,y_test_predicted,average='macro'), f'Acc: {accuracy_score(y_test, y_test_predicted)}')

F1 Score of train data: 0.8611 Acc: 0.8611385324119776
F1 Score of test data: 0.8176 Acc: 0.8177028451001054


## Ensemble of the models 1: Max Voting

The selected models are SVC, Random forest and Xgboost classifiers.

In [35]:
from sklearn.metrics import log_loss
from sklearn.ensemble import VotingClassifier

model_1 = svm.SVC(C= 10, kernel= 'rbf', probability=True)
model_2 = RandomForestClassifier(max_depth= 50, n_estimators=1000)
model_3 = xgb.XGBClassifier()
final_model = VotingClassifier(estimators=[('svc', model_1), ('rf', model_2), ('xgb', model_3)], voting='hard')
final_model.fit(X_train, y_train)
y_train_predicted = final_model.predict(X_train)
y_test_predicted = final_model.predict(X_test)

# printing log loss between actual and predicted value
print(log_loss(y_test, y_test_predicted))
print('F1 Score of train data: %0.4f' %f1_score(y_train,y_train_predicted,average='macro'), f'Acc: {accuracy_score(y_train, y_train_predicted)}')
print('F1 Score of test data: %0.4f' %f1_score(y_test,y_test_predicted,average='macro'), f'Acc: {accuracy_score(y_test, y_test_predicted)}')

INFO:root:sklearn.svm.SVC.fit: running accelerated version on CPU
INFO:root:sklearn.svm.SVC.fit: running accelerated version on CPU
INFO:root:sklearn.ensemble.RandomForestClassifier.fit: running accelerated version on CPU
INFO:root:sklearn.svm.SVC.predict: running accelerated version on CPU
INFO:root:sklearn.ensemble.RandomForestClassifier.predict: running accelerated version on CPU
INFO:root:sklearn.svm.SVC.predict: running accelerated version on CPU
INFO:root:sklearn.ensemble.RandomForestClassifier.predict: running accelerated version on CPU


5.723168762478601
F1 Score of train data: 0.9930 Acc: 0.9929582099374794
F1 Score of test data: 0.8342 Acc: 0.8342992623814541


In [36]:
from sklearn.metrics import log_loss
from sklearn.ensemble import VotingClassifier

model_1 = svm.SVC(C= 10, kernel= 'rbf', probability=True)
model_2 = RandomForestClassifier(max_depth= 50, n_estimators=1000)
model_3 = xgb.XGBClassifier()
final_model = VotingClassifier(estimators=[('svc', model_1), ('rf', model_2), ('xgb', model_3)], voting='soft')
final_model.fit(X_train, y_train)
y_train_predicted = final_model.predict(X_train)
y_test_predicted = final_model.predict(X_test)

# printing log loss between actual and predicted value
print(log_loss(y_test, y_test_predicted))
print('F1 Score of train data: %0.4f' %f1_score(y_train,y_train_predicted,average='macro'), f'Acc: {accuracy_score(y_train, y_train_predicted)}')
print('F1 Score of test data: %0.4f' %f1_score(y_test,y_test_predicted,average='macro'), f'Acc: {accuracy_score(y_test, y_test_predicted)}')

INFO:root:sklearn.svm.SVC.fit: running accelerated version on CPU
INFO:root:sklearn.svm.SVC.fit: running accelerated version on CPU
INFO:root:sklearn.ensemble.RandomForestClassifier.fit: running accelerated version on CPU
INFO:root:sklearn.svm.SVC.predict_proba: running accelerated version on CPU
INFO:root:sklearn.svm.SVC.decision_function: running accelerated version on CPU
INFO:root:sklearn.ensemble.RandomForestClassifier.predict_proba: running accelerated version on CPU
INFO:root:sklearn.svm.SVC.predict_proba: running accelerated version on CPU
INFO:root:sklearn.svm.SVC.decision_function: running accelerated version on CPU
INFO:root:sklearn.ensemble.RandomForestClassifier.predict_proba: running accelerated version on CPU


5.777760927592414
F1 Score of train data: 0.9875 Acc: 0.98749588680487
F1 Score of test data: 0.8327 Acc: 0.8327186512118019


In [37]:
from sklearn.metrics import log_loss
from sklearn.ensemble import VotingClassifier

model_1 = svm.SVC(C= 10, kernel= 'rbf', probability=True)
model_2 = RandomForestClassifier(max_depth= 50, n_estimators=1000)
model_3 = xgb.XGBClassifier()
final_model = VotingClassifier(estimators=[('svc', model_1), ('rf', model_2), ('xgb', model_3)], voting='hard', weights=[0.85,0.83,0.83])
final_model.fit(X_train, y_train)
y_train_predicted = final_model.predict(X_train)
y_test_predicted = final_model.predict(X_test)

# printing log loss between actual and predicted value
print(log_loss(y_test, y_test_predicted))
print('F1 Score of train data: %0.4f' %f1_score(y_train,y_train_predicted,average='macro'), f'Acc: {accuracy_score(y_train, y_train_predicted)}')
print('F1 Score of test data: %0.4f' %f1_score(y_test,y_test_predicted,average='macro'), f'Acc: {accuracy_score(y_test, y_test_predicted)}')

INFO:root:sklearn.svm.SVC.fit: running accelerated version on CPU
INFO:root:sklearn.svm.SVC.fit: running accelerated version on CPU
INFO:root:sklearn.ensemble.RandomForestClassifier.fit: running accelerated version on CPU
INFO:root:sklearn.svm.SVC.predict: running accelerated version on CPU
INFO:root:sklearn.ensemble.RandomForestClassifier.predict: running accelerated version on CPU
INFO:root:sklearn.svm.SVC.predict: running accelerated version on CPU
INFO:root:sklearn.ensemble.RandomForestClassifier.predict: running accelerated version on CPU


5.677675326657442
F1 Score of train data: 0.9930 Acc: 0.9929582099374794
F1 Score of test data: 0.8355 Acc: 0.8356164383561644


In [38]:
from sklearn.metrics import log_loss
from sklearn.ensemble import VotingClassifier

model_1 = svm.SVC(C= 10, kernel= 'rbf', probability=True)
model_2 = RandomForestClassifier(max_depth= 50, n_estimators=1000)
model_3 = xgb.XGBClassifier()
final_model = VotingClassifier(estimators=[('svc', model_1), ('rf', model_2), ('xgb', model_3)], voting='soft', weights=[0.85,0.83,0.83])
final_model.fit(X_train, y_train)
y_train_predicted = final_model.predict(X_train)
y_test_predicted = final_model.predict(X_test)

# printing log loss between actual and predicted value
print(log_loss(y_test, y_test_predicted))
print('F1 Score of train data: %0.4f' %f1_score(y_train,y_train_predicted,average='macro'), f'Acc: {accuracy_score(y_train, y_train_predicted)}')
print('F1 Score of test data: %0.4f' %f1_score(y_test,y_test_predicted,average='macro'), f'Acc: {accuracy_score(y_test, y_test_predicted)}')

INFO:root:sklearn.svm.SVC.fit: running accelerated version on CPU
INFO:root:sklearn.svm.SVC.fit: running accelerated version on CPU
INFO:root:sklearn.ensemble.RandomForestClassifier.fit: running accelerated version on CPU
INFO:root:sklearn.svm.SVC.predict_proba: running accelerated version on CPU
INFO:root:sklearn.svm.SVC.decision_function: running accelerated version on CPU
INFO:root:sklearn.ensemble.RandomForestClassifier.predict_proba: running accelerated version on CPU
INFO:root:sklearn.svm.SVC.predict_proba: running accelerated version on CPU
INFO:root:sklearn.svm.SVC.decision_function: running accelerated version on CPU
INFO:root:sklearn.ensemble.RandomForestClassifier.predict_proba: running accelerated version on CPU


5.814156476689342
F1 Score of train data: 0.9866 Acc: 0.9865745310957552
F1 Score of test data: 0.8316 Acc: 0.8316649104320337
